In [0]:
"""
Vector Search Cleanup
=============================
Deletes vector search indexes and optionally the endpoint to save costs.

USAGE:
------
Set DELETE_ENDPOINT flag below to control cleanup behavior:
- True: Delete indexes AND endpoint (full cleanup, maximum cost savings)
- False: Delete indexes only (keep endpoint for faster recreation)
"""

import sys
import logging
from databricks.vector_search.client import VectorSearchClient

# ============================================================================
# CONFIGURATION - Set this flag to control cleanup behavior
# ============================================================================
DELETE_ENDPOINT = True  # Change to False to keep endpoint and delete only indexes

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

def delete_vector_index(client: VectorSearchClient, index_name: str):
    """Delete a vector search index."""
    try:
        logger.info(f"Attempting to delete index: {index_name}")
        client.delete_index(index_name=index_name)
        logger.info(f"Successfully deleted index: {index_name}")
    except Exception as e:
        if "does not exist" in str(e).lower() or "not found" in str(e).lower():
            logger.info(f"Index {index_name} does not exist, skipping")
        else:
            logger.error(f"Error deleting index {index_name}: {e}")
            raise

def delete_endpoint(client: VectorSearchClient, endpoint_name: str):
    """Delete a vector search endpoint."""
    try:
        logger.info(f"Attempting to delete endpoint: {endpoint_name}")
        client.delete_endpoint(name=endpoint_name)
        logger.info(f"Successfully deleted endpoint: {endpoint_name}")
    except Exception as e:
        if "does not exist" in str(e).lower() or "not found" in str(e).lower():
            logger.info(f"Endpoint {endpoint_name} does not exist, skipping")
        else:
            logger.error(f"Error deleting endpoint {endpoint_name}: {e}")
            raise

def main(delete_endpoint_flag: bool = True):
    """Clean up vector search resources to save costs.
    
    Args:
        delete_endpoint_flag: If True, delete both indexes and endpoint
                             If False, delete only indexes (keep endpoint)
    """
    
    # Configuration - should match 00_ConfigLab.py
    catalog = "ai_pioneer"
    schema = "lab_data"
    endpoint_name = "ai_pioneer_vs_endpoint"
    
    # Index names
    kb_index = f"{catalog}.{schema}.knowledge_base_index"
    st_index = f"{catalog}.{schema}.support_tickets_index"
    
    logger.info("=" * 60)
    if delete_endpoint_flag:
        logger.info("VECTOR SEARCH CLEANUP - Deleting indexes and endpoint")
    else:
        logger.info("VECTOR SEARCH CLEANUP - Deleting indexes only")
    logger.info("=" * 60)
    
    # Initialize client
    vs_client = VectorSearchClient()
    
    # Step 1: Delete indexes
    logger.info("Step 1: Deleting vector search indexes...")
    delete_vector_index(vs_client, kb_index)
    delete_vector_index(vs_client, st_index)
    
    # Step 2: Delete endpoint (optional)
    if delete_endpoint_flag:
        logger.info("Step 2: Deleting vector search endpoint...")
        delete_endpoint(vs_client, endpoint_name)
        logger.info("=" * 60)
        logger.info("CLEANUP COMPLETE - All vector search resources deleted")
        logger.info("Cost savings: Vector search endpoint no longer incurring charges")
    else:
        logger.info("Step 2: Skipping endpoint deletion (endpoint preserved)")
        logger.info("=" * 60)
        logger.info("CLEANUP COMPLETE - Indexes deleted, endpoint preserved")
        logger.info("Cost savings: Indexes removed, endpoint still available for reuse")
    
    logger.info("=" * 60)
    logger.info("")
    logger.info("NOTE: Your data tables and volumes are still intact.")
    if delete_endpoint_flag:
        logger.info("To recreate the vector search resources, run 00_ConfigLab.py again.")
    else:
        logger.info("To recreate indexes, run 00_ConfigLab.py again (endpoint reuse is faster).")

if __name__ == "__main__":
    try:
        main(delete_endpoint_flag=DELETE_ENDPOINT)
    except Exception as e:
        logger.error(f"Cleanup script failed: {e}")
        sys.exit(1)

